In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
import tensorflow as tf
import matplotlib.pyplot as plt

# **Load Data**

In [ ]:
data_path = '/kaggle/input/wild-animals/Animals'  

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(64, 64),  
    batch_size=32
)


In [ ]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(64, 64),  
    batch_size=32
)

In [ ]:
class_names = train_data.class_names
print(f"Classes: {class_names}")

# Convert Image to vector array

In [ ]:
def dataset_to_numpy(data):
    images, labels = [], []
    for image_batch, label_batch in data:
        images.append(image_batch.numpy())
        labels.append(label_batch.numpy())
    return np.concatenate(images), np.concatenate(labels)

In [ ]:
X_train, y_train = dataset_to_numpy(train_data)
X_test, y_test = dataset_to_numpy(test_data)

In [ ]:
X_train=X_train.reshape(X_train.shape[0],-1)
X_test=X_test.reshape(X_test.shape[0],-1)

# **Normalize Data**

In [ ]:
X_train=StandardScaler().fit_transform(X_train)
X_test=StandardScaler().fit_transform(X_test)

In [2]:
#Flatten the Array 
# so if ndarray is (5,1) the ravel() function will make it (5,)
y_train=y_train.ravel()
y_test=y_test.ravel() 

NameError: name 'y_train' is not defined

# **Apply K-Fold cross validation**

In [ ]:
num=5
kfold=KFold(n_splits=num,shuffle=True)

In [ ]:
Manhattan_distance=[]
Euclidean_distrance=[]

# **Calculate Manhattan and Euclidean Distance**

In [ ]:
for fold,(train_index,test_index) in enumerate(kfold.split(X_train)):
  print(f"Fold {fold + 1}:")

  X_fold_train,X_fold_val=X_train[train_index],X_train[test_index]
  y_fold_train,y_fold_val=y_train[train_index],y_train[test_index]

  knn_manhattan=KNeighborsClassifier(n_neighbors=10,p=1)
  knn_euclidean=KNeighborsClassifier(n_neighbors=10,p=2)

  knn_manhattan.fit(X_fold_train,y_fold_train)
  knn_euclidean.fit(X_fold_train,y_fold_train)

  pred_manhattan=knn_manhattan.predict(X_fold_val)
  pred_euclidean=knn_euclidean.predict(X_fold_val)

  accuracy_manhattan=np.mean(pred_manhattan==y_fold_val)
  accuracy_euclidean=np.mean(pred_euclidean==y_fold_val)

  Manhattan_distance.append(accuracy_manhattan)
  Euclidean_distrance.append(accuracy_euclidean)
  print(f'Manhattan validation accuracy: {accuracy_manhattan}')
  print(f'Euclidean validation accuracy: {accuracy_euclidean}')




In [ ]:
avg_manhattan=np.mean(Manhattan_distance)
avg_euclidean=np.mean(Euclidean_distrance)

print(f'Average Manhattan validation accuracy: {avg_manhattan}')
print(f'Average Euclidean validation accuracy: {avg_euclidean}')

In [ ]:
knn_manhattan.fit(X_train, y_train)
knn_euclidean.fit(X_train, y_train)

# **Predict**

In [ ]:
predictions_manhattan_test = knn_manhattan.predict(X_test)
predictions_euclidean_test = knn_euclidean.predict(X_test)

In [ ]:
test_accuracy_manhattan = np.mean(predictions_manhattan_test == y_test)
test_accuracy_euclidean = np.mean(predictions_euclidean_test == y_test)
print(f'Test accuracy (Manhattan): {test_accuracy_manhattan}')
print(f'Test accuracy (Euclidean): {test_accuracy_euclidean}')

# **Visualize**

In [ ]:
def plot_images_with_predictions(images, predictions, class_names):
    plt.figure(figsize=(10, 5))
    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(images[i])
        plt.title(f"Predicted: {class_names[predictions[i]]}")
        plt.axis('off')
    plt.show()

class_names = ['Cheetah', 'Jaguar', 'Leopard', 'Lion', 'Tiger']


plot_images_with_predictions(X_test[:5].reshape(-1, 32, 32, 3), predictions_manhattan_test[:5], class_names)
plot_images_with_predictions(X_test[:5].reshape(-1, 32, 32, 3), predictions_euclidean_test[:5], class_names)
